In [92]:
!pip install altair

Note: you may need to restart the kernel to use updated packages.


In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
import altair as alt 

%matplotlib inline

In [79]:
##response per country
df_survey_responses = pd.read_csv(ZipFile("../data/kaggle_survey/kaggle_survey_2020_responses.csv.zip").open("kaggle_survey_2020_responses.csv"))
df_multi_choice = pd.read_csv(ZipFile("../data/kaggle_survey/multiple_choice_responses.csv.zip").open("multiple_choice_responses.csv"))


C:\Users\zhang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\zhang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,3,7,19,34,47,49,50,51,52,53,54,68,81,94,96,109,115,130,139,147,154,167,180,193,206,219,232,245) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [80]:
df_survey_responses.head()

,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,...,Q35_B_Part_2,Q35_B_Part_3,Q35_B_Part_4,Q35_B_Part_5,Q35_B_Part_6,Q35_B_Part_7,Q35_B_Part_8,Q35_B_Part_9,Q35_B_Part_10,Q35_B_OTHER
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,For how many years have you been writing code ...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,...,"In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor..."
1,1838,35-39,Man,Colombia,Doctoral degree,Student,5-10 years,Python,R,SQL,...,NaN,NaN,NaN,TensorBoard,NaN,NaN,NaN,NaN,NaN,NaN
2,289287,30-34,Man,United States of America,Master’s degree,Data Engineer,5-10 years,Python,R,SQL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,860,35-39,Man,Argentina,Bachelor’s degree,Software Engineer,10-20 years,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
4,507,30-34,Man,United States of America,Master’s degree,Data Scientist,5-10 years,Python,NaN,SQL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
df_multi_choice.head()

,Time from Start to Finish (seconds),Q1,Q2,Q2_OTHER_TEXT,Q3,Q4,Q5,Q5_OTHER_TEXT,Q6,Q7,...,Q34_Part_4,Q34_Part_5,Q34_Part_6,Q34_Part_7,Q34_Part_8,Q34_Part_9,Q34_Part_10,Q34_Part_11,Q34_Part_12,Q34_OTHER_TEXT
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,What is your gender? - Prefer to self-describe...,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,Select the title most similar to your current ...,What is the size of the company where you are ...,Approximately how many individuals are respons...,...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...
1,510,22-24,Male,-1,France,Master’s degree,Software Engineer,-1,"1000-9,999 employees",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2,423,40-44,Male,-1,India,Professional degree,Software Engineer,-1,"> 10,000 employees",20+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
3,83,55-59,Female,-1,Germany,Professional degree,NaN,-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
4,391,40-44,Male,-1,Australia,Master’s degree,Other,0,"> 10,000 employees",20+,...,NaN,NaN,NaN,NaN,NaN,Azure SQL Database,NaN,NaN,NaN,-1


In [82]:
def abbreviate_nations(df):
    df = df.drop(0, axis=0)
    #shorten longest country names
    df.loc[df['Q3']=="United Kingdom of Great Britain and Northern Ireland", "Q3"] = "United Kingdom"
    df.loc[df['Q3']=="Iran, Islamic Republic of...", "Q3"] = "Iran"
    
    return df

In [83]:
col_list = df_survey_responses.columns.to_list()
col_list
#create a mapping dict from question id to question content
question_mapping = df_survey_responses.loc[0].to_dict()
print(question_mapping)

df_survey_responses = abbreviate_nations(df_survey_responses)
df_multi_choice = abbreviate_nations(df_multi_choice)

{'Time from Start to Finish (seconds)': 'Duration (in seconds)', 'Q1': 'What is your age (# years)?', 'Q2': 'What is your gender? - Selected Choice', 'Q3': 'In which country do you currently reside?', 'Q4': 'What is the highest level of formal education that you have attained or plan to attain within the next 2 years?', 'Q5': 'Select the title most similar to your current role (or most recent title if retired): - Selected Choice', 'Q6': 'For how many years have you been writing code and/or programming?', 'Q7_Part_1': 'What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python', 'Q7_Part_2': 'What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - R', 'Q7_Part_3': 'What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - SQL', 'Q7_Part_4': 'What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C', 'Q7

In [84]:
country_2019 = df_multi_choice.groupby(['Q3'])
df_country_2019 = country_2019.agg('count')
df_country_2019.reset_index(inplace=True)
df_country_2019=df_country_2019[['Q3', 'Q1']]
df_country_2019.columns=('Country', 'Count')
df_country_2019['year'] = 2019
df_country_2019.tail()

,Country,Count,year
54,Turkey,288,2019
55,Ukraine,191,2019
56,United Kingdom,482,2019
57,United States of America,3085,2019
58,Viet Nam,128,2019


In [106]:
def chart_country(df, sort=None):
    chart_country_groups = alt.Chart(df_country_2019).mark_bar().encode(
    alt.X('Country:N', sort=sort, axis=alt.Axis(grid=False)),
    alt.Y('Count:Q', title="Count of Responses", axis=alt.Axis(grid=False))
    ).properties(width=680,
    background='#f5f5f5',title={
"text": ["Number of Respondents per Country"],
"fontSize":20, "fontWeight":600, "subtitleFontSize":14
}
    )
    
    return chart_country_groups

chart_country_groups = chart_country(df_country_2019)

In [107]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#labels = ['G1', 'G2', 'G3', 'G4', 'G5']
#year_2019 = df_country_comparison.loc[df_country_comparison['year']==2019, "C"]
#women_means = [25, 32, 34, 20, 25]

x = np.arange(len(df_country_count_by_year['Country']))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(15, 6))

rects1 = ax.bar(x - width/2, df_country_count_by_year['Count_2019'], width, label='2019')
rects2 = ax.bar(x + width/2, df_country_count_by_year['Count'], width, label='2020')

 # Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Responses', fontsize=14)
ax.set_xlabel('Country', fontsize=14)
ax.set_title('Survey Responses by Country and Year', fontsize=18)
ax.set_xticks(x)
ax.set_xticklabels(df_country_count_by_year['Country'])
ax.set_facecolor('k')
ax.patch.set_alpha(0.25)
plt.xticks(rotation=90)
fig.patch.set_alpha(0.5)
ax.legend()
ax.annotate('Responses from India increased', xy=(0.5, 4250), xytext=(3, 4500),
            arrowprops=dict(facecolor='black'),
            )
ax.annotate('Responses from the USA decreased', xy=(1.5, 2000), xytext=(4, 2250),
            arrowprops=dict(facecolor='black'),
            )



#fig.tight_layout()
plt.show()

NameError: name 'df_country_count_by_year' is not defined

In [101]:
df_country_count_by_year['yoy_change_pct'] = 100 * (df_country_count_by_year['Count'] - df_country_count_by_year['Count_2019']) / df_country_count_by_year['Count_2019']

NameError: name 'df_country_count_by_year' is not defined

In [102]:
country_alterations ={'Nepal': 'NPL', 'Sri Lanka':'LKA', 'United Arab Emirates':'ARE', 'Ghana':'GHA'}
country_codes.update(country_alterations)
country_codes

NameError: name 'country_codes' is not defined

In [7]:
df_country_count_by_year['country_code'] = df_country_count_by_year['Country']
df_country_count_by_year.replace({"country_code": country_codes}, inplace=True)
df_country_count_by_year['yoy_change_pct'] = np.round(df_country_count_by_year['yoy_change_pct'],1)
df_country_count_by_year

NameError: name 'df_country_count_by_year' is not defined

In [8]:
alt.Chart(df_age_country).mark_rect().encode(
    alt.X('Country:N', sort=None, title="Country sorted by descending count of aggregate responses"),
    alt.Y('Age:N'),
    color=alt.Color('LogCount:Q', legend=alt.Legend(title="Log Count of Respondents", orient="top"))
).properties(width=680,
    background='#f5f5f5',title={
"text": ["Age of Respondents"],
"subtitle": ["Broken Down by Country"],
"fontSize":20, "fontWeight":600, "subtitleFontSize":14
}
)

NameError: name 'alt' is not defined

In [9]:
df_age_country['age_group_pct'] = 100 * df_age_country['Count'] / df_age_country['Count_total']
df_age_country

NameError: name 'df_age_country' is not defined

In [10]:
alt.Chart(df_age_country).mark_rect().encode(
    alt.X('Country:N', sort=None, title="Country sorted by descending count of aggregate responses"),
    alt.Y('Age:N'),
    color=alt.Color('age_group_pct:Q', legend=alt.Legend(title="% of Respondents", orient="top"))
).properties(width=680,
    background='#f5f5f5',title={
"text": ["Age of Respondents"],
"subtitle": ["Broken Down proportionally by Country"],
"fontSize":20, "fontWeight":600, "subtitleFontSize":14
}
)

NameError: name 'alt' is not defined

In [11]:
df_age_country_under_25 = df_age_country.loc[df_age_country['Age'].isin(['18-21', '22-24'])]

under_25 = df_age_country_under_25.groupby(['Country'])
df_under_25 = under_25.agg('sum')
df_under_25.reset_index(inplace=True)
df_under_25=df_under_25[['Country', 'age_group_pct']]
df_under_25.columns=('Country', 'pct_under_25')

df_under_25['pct_under_25'] = round(df_under_25['pct_under_25'],1)

df_under_25['country_code'] = df_under_25['Country']
df_under_25.replace({"country_code": country_codes}, inplace=True)

df_under_25

NameError: name 'df_age_country' is not defined

In [12]:
df_age_country_under_25 = df_age_country.loc[df_age_country['Age'].isin(['18-21', '22-24'])]

under_25 = df_age_country_under_25.groupby(['Country'])
df_under_25 = under_25.agg('sum')
df_under_25.reset_index(inplace=True)
df_under_25=df_under_25[['Country', 'age_group_pct']]
df_under_25.columns=('Country', 'pct_under_25')

df_under_25['pct_under_25'] = round(df_under_25['pct_under_25'],1)

df_under_25['country_code'] = df_under_25['Country']
df_under_25.replace({"country_code": country_codes}, inplace=True)

df_under_25

NameError: name 'df_age_country' is not defined

In [13]:
sive and it may be better to consider using matplotlib/seaborn
# altair usually limits input sources to 5000 rows but I have overriden that setting
# the code commented below produces the chart below, but the chart is loaded from a saved image that uses less memory

# alt.Chart(df_survey_responses).transform_density(
#     'Time from Start to Finish (seconds)',
#     as_=['Time from Start to Finish (seconds)', 'density'],
#     extent=[0, 3000],
#     groupby=['Q1']
# ).mark_area(orient='horizontal').encode(
#     y=alt.Y('Time from Start to Finish (seconds):Q', title=''),
#     color=alt.Color('Q1:N', legend=alt.Legend(title="Age", orient="top")),
#     x=alt.X(
#         'density:Q',
#         stack='center',
#         impute=None,
#         title=None,
#         axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
#     ),
#     column=alt.Column(
#         'Q1:N',
#         header=alt.Header(
#             titleOrient='bottom',
#             labelOrient='bottom',
#             labelPadding=0,
#         ),
#     )
# ).properties(width=80,
#     background='#f5f5f5',title={
# "text": ["Survey Response Time in Seconds"],
# "subtitle": ["Broken down by Age"],
# "fontSize":20, "fontWeight":600, "subtitleFontSize":14
# }
# ).configure_facet(
#     spacing=0
# ).configure_view(
#     stroke=None
# )


SyntaxError: invalid syntax (<ipython-input-13-ef94fcee56b5>, line 1)

In [14]:
#only top countries

alt.Chart(df_age_country[df_age_country['Country'].isin(country_sort[:9])]).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
    y=alt.Y('Age:O'),
    color=alt.Color('Country:N', legend=alt.Legend(orient="right"))
).properties(width=320,
             background='#f5f5f5',title={
"text": ["Age of Respondents"],
"subtitle": ["Broken Down proportionally by Country"],
"fontSize":20, "fontWeight":600, "subtitleFontSize":14
})

NameError: name 'alt' is not defined

In [15]:
education_country = df_survey_responses.groupby(['Q4', 'Q3'])
df_education_country = education_country.agg('count')
df_education_country.reset_index(inplace=True)
df_education_country=df_education_country[['Q4', 'Q3', 'Q2']]
df_education_country.columns=('education', 'country', 'Count')
df_education_country['rank'] = df_education_country['education']
education_sort_map = {"Doctoral degree": 6, 'Master’s degree': 5, "Professional degree": 4, 'Bachelor’s degree': 3, 
                  "Some college/university study without earning a bachelor’s degree": 2,
                  "No formal education past high school": 1, "I prefer not to answer": 0
                 }
df_education_country['rank'] = df_education_country['rank'].replace(education_sort_map)
df_education_country['education_rank'] = df_education_country['rank'].astype(str) + ' - ' + df_education_country['education']

NameError: name 'df_survey_responses' is not defined

In [16]:
alt.Chart(df_education_country).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('country:N'),
    y=alt.Y('Count:Q', stack="normalize", title="Normalized Count"),
    color=alt.Color('education_rank:N', legend=alt.Legend(orient="left"))
).properties(width=520,
             background='#f5f5f5',title={
"text": ["Highest Educational Achievement Held by Respondents"],
"subtitle": ["Broken Down proportionally by Country"],
"fontSize":18, "fontWeight":600, "subtitleFontSize":14
})

NameError: name 'alt' is not defined

In [17]:
def platform_altair(df, colorfeat, colortitle):
    chrt = alt.Chart(df_ed_platform_full).mark_rect().encode(
        x=alt.X('country:N'),
        y=alt.Y('Platform:N'),
        color=alt.Color(colorfeat, legend=alt.Legend(orient="bottom"), title=colortitle)
    ).properties(width=600, height=200,
                 background='#f5f5f5',title={
    "text": ["Online Platforms Used by Respondents"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })
    
    return chrt

In [18]:
chrt = platform_altair(df_ed_platform_full, 'Count:Q', 'Count')
chrt

NameError: name 'df_ed_platform_full' is not defined

In [19]:
log_chrt = platform_altair(df_ed_platform_full, 'logCount:Q', 'Log Count')
log_chrt

NameError: name 'df_ed_platform_full' is not defined

In [20]:
df_ed_platform_full = df_ed_platform_full.merge(df_country, left_on='country', right_on='Country',
          suffixes=(None, '_total'))
df_ed_platform_full['platform_pct'] = round(100 * df_ed_platform_full['Count'] / df_ed_platform_full['Count_total'],1)
df_ed_platform_full

NameError: name 'df_ed_platform_full' is not defined

In [21]:
pct_chrt = platform_altair(df_ed_platform_full, 'platform_pct:Q', '% in Country')
pct_chrt

NameError: name 'df_ed_platform_full' is not defined

In [22]:
df_ed_platform_full.loc[df_ed_platform_full['Country']=='Belarus']

NameError: name 'df_ed_platform_full' is not defined

In [23]:
alt.Chart(df_job_country).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('country:N'),
    y=alt.Y('Count:Q', stack="normalize", title="Normalized Count"),
    color=alt.Color('job:N', legend=alt.Legend(orient="left"))
).properties(width=550,
             background='#f5f5f5',title={
"text": ["Job Titles Held by Respondents"],
"subtitle": ["Broken Down proportionally by Country"],
"fontSize":18, "fontWeight":600, "subtitleFontSize":14
})

NameError: name 'alt' is not defined

In [24]:
df_job_proportions = df_job_country.merge(df_country, left_on='country', right_on='Country',
          suffixes=(None, '_total'))
df_job_proportions.drop(columns=['country'], inplace=True)
df_job_proportions['job_proportion'] = df_job_proportions['Count'] / df_job_proportions['Count_total']
df_job_proportions

NameError: name 'df_job_country' is not defined

In [25]:
df_job_proportions_data_scientists = df_job_proportions.loc[df_job_proportions['job']=="Data Scientist"]
df_job_proportions_data_scientists

NameError: name 'df_job_proportions' is not defined

In [26]:
alt.Chart(df_experience_country).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
    x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
    color=alt.Color('experience_rank:N', legend=alt.Legend(orient="bottom"), title="Years of Experience")
).properties(width=480, height=520,
             background='#f5f5f5',title={
"text": ["Years of Experience"],
"subtitle": ["Broken Down proportionally by Country"],
"fontSize":18, "fontWeight":600, "subtitleFontSize":14
})

NameError: name 'alt' is not defined

In [27]:
prog_lang_country = df_survey_responses.groupby(['Q8', 'Q3'])
df_prog_lang_country = prog_lang_country.agg('count')
df_prog_lang_country.reset_index(inplace=True)
df_prog_lang_country=df_prog_lang_country[['Q8', 'Q3', 'Q2']]
df_prog_lang_country.columns=('prog_lang', 'country', 'Count')
#sort by count
df_prog_lang_country = df_prog_lang_country.sort_values(by=['Count'], ascending=False)
df_prog_lang_country.head()

NameError: name 'df_survey_responses' is not defined

In [28]:
def prog_lang_country(df, title):
    chart_prog_lang_country = alt.Chart(df).mark_bar(
        cornerRadiusTopLeft=3,
        cornerRadiusTopRight=3
    ).encode(
        y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
        x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
        color=alt.Color('prog_lang:N', legend=alt.Legend(orient="bottom"), title="Programming Language")
    ).properties(width=480, height=500,
                 background='#f5f5f5',title={
    "text": [title],
    "subtitle": ["Broken Down proportionally by Country"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })
    
    return chart_prog_lang_country

In [29]:
prog_lang_country(df_prog_lang_country, "Recommended Starting Programming Language for Data Science")

NameError: name 'df_prog_lang_country' is not defined

In [30]:
prog_lang_country(df_prog_lang_country.loc[df_prog_lang_country['prog_lang']!="Python"], "Recommended Starting Programming Language for Data Science (excluding Python)")

NameError: name 'df_prog_lang_country' is not defined

In [31]:
platform_country = df_survey_responses.groupby(['Q11', 'Q3'])
df_platform_country = platform_country.agg('count')
df_platform_country.reset_index(inplace=True)
df_platform_country=df_platform_country[['Q11', 'Q3', 'Q2']]
df_platform_country.columns=('platform', 'country', 'Count')
#sort by count
df_platform_country = df_platform_country.sort_values(by=['Count'], ascending=False)
df_platform_country.head()

NameError: name 'df_survey_responses' is not defined

In [32]:
def platform_country_chart(df, title):
    chart_platform_country = alt.Chart(df).mark_bar(
        cornerRadiusTopLeft=3,
        cornerRadiusTopRight=3
    ).encode(
        y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
        x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
        color=alt.Color('platform:N', legend=alt.Legend(orient="top"), title="Platform")
    ).properties(width=400, height=500,
                 background='#f5f5f5',title={
    "text": [title],
    "subtitle": ["Broken Down proportionally by Country"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })
    
    return chart_platform_country

In [33]:
df_platform_country = df_platform_country.loc[df_platform_country['platform']!="A personal computer or laptop"]
alt.themes.enable('vox')
platform_country_chart(df_platform_country, "Preferred Platform for Data Science (excluding pc's)")

NameError: name 'df_platform_country' is not defined

In [34]:
alt.themes.enable('latimes')
bars = alt.Chart(df_tpu_country).mark_bar(
        cornerRadiusTopLeft=3,
        cornerRadiusTopRight=3
    ).encode(
        y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
        x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
        color=alt.Color('tpu_usage_rank:N', legend=alt.Legend(orient="top"), title="TPU Usage")
    ).properties(width=400, height=500,title={
    "text": ["TPU Usage"],
    "subtitle": ["Broken Down proportionally by Country"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })

bars

NameError: name 'alt' is not defined

In [35]:
ml_country = df_survey_responses.groupby(['Q15', 'Q3'])
df_ml_country = ml_country.agg('count')
df_ml_country.reset_index(inplace=True)
df_ml_country=df_ml_country[['Q15', 'Q3', 'Q2']]
df_ml_country.columns=('ml_years', 'country', 'Count')
#sort by count
df_ml_country = df_ml_country.sort_values(by=['Count'], ascending=False)
df_ml_country.head()

NameError: name 'df_survey_responses' is not defined

In [36]:
bars_ml = alt.Chart(df_ml_country).mark_bar(
        cornerRadiusTopLeft=3,
        cornerRadiusTopRight=3
    ).encode(
        y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
        x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
        color=alt.Color('ml_years_rank:N', legend=alt.Legend(orient="top"), title="Years using ML methods")
    ).properties(width=500, height=500,title={
    "text": ["Years using ML methods"],
    "subtitle": ["Broken Down proportionally by Country"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })

bars_ml

NameError: name 'alt' is not defined

In [37]:
df_ml_country['ML_experience_freq_pct'] = 100 * df_ml_country['Count'] / df_ml_country['Count_total']
df_ml_country

NameError: name 'df_ml_country' is not defined

In [38]:
df_ml_country_5_plus = df_ml_country[df_ml_country['ml_years_rank'].isin(['6 - 5-10 years', '7 - 10-20 years','8 - 20 or more years'])]
df_ml_country_5_plus

NameError: name 'df_ml_country' is not defined

In [39]:
df_ml_5_plus = df_ml_country_5_plus.groupby(['country_code'])
df_ml_5_plus = df_ml_5_plus.agg('ML_experience_freq_pct').sum().to_frame().reset_index()
df_ml_5_plus['ML_experience_freq_pct'] = np.round(df_ml_5_plus['ML_experience_freq_pct'],1)
df_ml_5_plus

NameError: name 'df_ml_country_5_plus' is not defined

In [40]:
fig = px.choropleth(data_frame=df_ml_5_plus,
                    locations='country_code',
                    color='ML_experience_freq_pct',
                    color_continuous_scale="agsunset",
                    range_color=(0, 25),
                    hover_name='country_code', # column to add to hover information
                    title='% Respondents with 5+ years experience of ML Methods')
fig.show()

NameError: name 'px' is not defined

In [ ]:
##primary analysis
df_primary_analysis['primary_analysis_abbr'] = df_primary_analysis['primary_analysis']
primary_analysis_sort_map = {'Local development environments (RStudio, JupyterLab, etc.)': 'Local development environments',
                             'Basic statistical software (Microsoft Excel, Google Sheets, etc.)': 'Basic statistical software',
                            'Business intelligence software (Salesforce, Tableau, Spotfire, etc.)': 'Business intelligence software',
                            'Cloud-based data software & APIs (AWS, GCP, Azure, etc.)': 'Cloud-based data software & APIs',
                            'Advanced statistical software (SPSS, SAS, etc.)': 'Advanced statistical software',
                 }
df_primary_analysis['primary_analysis_abbr'] = df_primary_analysis['primary_analysis_abbr'].replace(primary_analysis_sort_map)
df_primary_analysi


In [41]:
def analysis_altair(df, colorfeat, colortitle):
    chrt = alt.Chart(df).mark_rect().encode(
        x=alt.X('country:N'),
        y=alt.Y('primary_analysis_abbr:N', title="Primary Analysis Toolset"),
        color=alt.Color(colorfeat, legend=alt.Legend(orient="bottom"), title=colortitle)
    ).properties(width=600, height=160,
                 background='#f5f5f5',title={
    "text": ["Primary Analysis Toolset Used by Respondents"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })
    
    return chrt

In [42]:
analysis_log_chrt = analysis_altair(df_primary_analysis, 'logCount:Q', 'Log Count')
analysis_log_chrt
#pct_chrt = platform_altair(df_primary_analysis, 'platform_pct:Q', '% in Country')
#pct_chrt

NameError: name 'df_primary_analysis' is not defined

In [43]:
df_primary_analysis = df_primary_analysis.merge(df_country, left_on='country', right_on='Country',
          suffixes=(None, '_total'))
df_primary_analysis['primary_analysis_pct'] = round(100 * df_primary_analysis['Count'] / df_primary_analysis['Count_total'],1)
df_primary_analysis = df_primary_analysis.sort_values(by=['primary_analysis_pct'], ascending=False)
df_primary_analysis

NameError: name 'df_primary_analysis' is not defined

In [44]:
analysis_pct_chrt = analysis_altair(df_primary_analysis, 'primary_analysis_pct:Q', '% by Country')
analysis_pct_chrt

NameError: name 'df_primary_analysis' is not defined

In [45]:
df_primary_analysis.loc[df_primary_analysis['Country']=='Japan']


NameError: name 'df_primary_analysis' is not defined

In [46]:
df_primary_analysis.loc[df_primary_analysis['Country']=='Japan']
df_primary_analysis_local = df_primary_analysis.loc[df_primary_analysis['primary_analysis_abbr']=='Local development environments']


NameError: name 'df_primary_analysis' is not defined

In [47]:
fig = px.choropleth(data_frame=df_primary_analysis_local,
                    locations='country_code',
                    color='primary_analysis_pct',
                    color_continuous_scale="agsunset",
                    range_color=(0, 50),
                    hover_name='Country', # column to add to hover information
                    title='% Respondents primarily using Local Development Environment')
fig.show()

NameError: name 'px' is not defined

In [48]:
company_size = df_survey_responses.groupby(['Q20', 'Q3'])
df_company_size = company_size.agg('count')
df_company_size.reset_index(inplace=True)
df_company_size=df_company_size[['Q20', 'Q3', 'Q2']]
df_company_size.columns=('company_size', 'country', 'Count')
df_company_size['rank'] = df_company_size['company_size']
company_size_sort_map = {"10,000 or more employees": 4, '1000-9,999 employees': 3, 
                  "250-999 employees": 2,
                  "50-249 employees": 1, "0-49 employees": 0
                 }
df_company_size['rank'] = df_company_size['rank'].replace(company_size_sort_map)
df_company_size['company_size_rank'] = df_company_size['rank'].astype(str) + ' - ' + df_company_size['company_size']

df_company_size

NameError: name 'df_survey_responses' is not defined

In [49]:
bars_company_size = alt.Chart(df_company_size).mark_bar(
        cornerRadiusTopLeft=3,
        cornerRadiusTopRight=3
    ).encode(
        y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
        x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
        color=alt.Color('company_size_rank:N', legend=alt.Legend(orient="bottom"), title="Company Size")
    ).properties(width=500, height=500,title={
    "text": ["Company Size of Respondent's Employer"],
    "subtitle": ["Broken Down proportionally by Country"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })

bars_company_size

NameError: name 'alt' is not defined

In [50]:
data_scientists = df_survey_responses.groupby(['Q21', 'Q3'])
df_data_scientists = data_scientists.agg('count')
df_data_scientists.reset_index(inplace=True)
df_data_scientists=df_data_scientists[['Q21', 'Q3', 'Q2']]
df_data_scientists.columns=('data_scientists', 'country', 'Count')

df_data_scientists['rank'] = df_data_scientists['data_scientists']
data_scientists_sort_map = {"20+": 6, "15-19": 5, "10-14": 4, '5-9': 3, 
                  "3-4": 2,
                  "1-2": 1, "0": 0
                 }
df_data_scientists['rank'] = df_data_scientists['rank'].replace(data_scientists_sort_map)
df_data_scientists['data_scientists_rank'] = df_data_scientists['rank'].astype(str) + ' - ' + df_data_scientists['data_scientists']

df_data_scientists

NameError: name 'df_survey_responses' is not defined

In [51]:
bubble_plot_company_size = alt.Chart(df_data_scientists).mark_circle(
        cornerRadiusTopLeft=3,
        cornerRadiusTopRight=3
    ).encode(
        y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
        x=alt.X('Count:Q', title="Count"),
        color=alt.Color('data_scientists_rank:N', legend=alt.Legend(orient="bottom"), title=""),
        size=alt.Size('Count:Q'),    
    ).properties(width=500, height=500,title={
    "text": ["Number of people responsible for Data Science workloads at place of Business"],
    "subtitle": ["Broken Down proportionally by Country"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })

bubble_plot_company_size

NameError: name 'alt' is not defined

In [52]:
ml = df_survey_responses.groupby(['Q22', 'Q3'])
df_ml = ml.agg('count')
df_ml.reset_index(inplace=True)
df_ml=df_ml[['Q22', 'Q3', 'Q2']]
df_ml.columns=('ml', 'country', 'Count')

df_ml['ml'].unique()

NameError: name 'df_survey_responses' is not defined

In [53]:
df_ml_prod = df_ml.loc[df_ml['ml'].isin(['We have well established ML methods (i.e., models in production for more than 2 years)',
       'We recently started using ML methods (i.e., models in production for less than 2 years)'])]
df_ml_prod = df_ml_prod.drop(columns=['ml'])

#get prod totals
ml_prod = df_ml_prod.groupby(['country'])
df_ml_prod = ml_prod.agg('sum').reset_index()

#get overall totals
ml_total = df_ml.groupby(['country'])
df_ml_total = ml_total.agg('sum').reset_index()


df_ml_prod = df_ml_prod.merge(df_ml_total[['country', 'Count']], left_on='country', right_on='country',
          suffixes=('_prod', '_total'))

df_ml_prod['country_code'] = df_ml_prod['country']
df_ml_prod.replace({"country_code": country_codes}, inplace=True)

df_ml_prod['ml_in_prod_pct'] = round(100 * df_ml_prod['Count_prod'] / df_ml_prod['Count_total'],1)
df_ml_prod

df_ml_prod_sorted = df_ml_prod.sort_values(by=['ml_in_prod_pct'], ascending=False)
df_ml_prod_sorted

NameError: name 'df_ml' is not defined

In [54]:
def ml_prod(df, subtitle, sort=None):
    ml_prod_chart = alt.Chart(df_ml_prod).mark_bar(
            cornerRadiusTopLeft=3,
            cornerRadiusTopRight=3
        ).encode(
            y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0), sort=sort),
            x=alt.X('ml_in_prod_pct:Q', title="%"),
        ).properties(width=200, height=500, title={
        "text": ["ML models in Production"],
        "subtitle": subtitle,        
        "fontSize":16, "fontWeight":600, "subtitleFontSize":12
        })
    
    return ml_prod_chart

bar_years_ml = ml_prod(df_ml_prod, "% whose place of Business Uses")
bar_years_ml_sorted = ml_prod(df_ml_prod_sorted, "% whose place of Business Uses", "-x")

bar_years_ml | bar_years_ml_sorted

NameError: name 'df_ml_prod' is not defined

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 246)

In [57]:
fig = px.choropleth(data_frame=df_ml_prod,
                    locations='country_code',
                    color='ml_in_prod_pct',
                    color_continuous_scale="turbo",
                    range_color=(10, 55),
                    hover_name='country', # column to add to hover information
                    title='% Respondents whose place of business uses ML in Production')
fig.show()

NameError: name 'px' is not defined

In [58]:
pay = df_survey_responses.groupby(['Q24', 'Q3'])
df_pay = pay.agg('count')
df_pay.reset_index(inplace=True)
df_pay=df_pay[['Q24', 'Q3', 'Q2']]
df_pay.columns=('pay', 'country', 'Count')

df_pay['pay'].unique()

NameError: name 'df_survey_responses' is not defined

In [59]:
df_pay['rank'] = df_pay['pay']
pay_sort_map = {"> $500,000": '24', "300,000-500,000": '23', "250,000-299,999": '22', '200,000-249,999': '21',
                "150,000-199,999": '20', "125,000-149,999": '19', "100,000-124,999": '18', '90,000-99,999': '17',
                "80,000-89,999": '16', "70,000-79,999": '15', "60,000-69,999": '14', '50,000-59,999': '13',
                "40,000-49,999": '12', "30,000-39,999": '11', "25,000-29,999": '10', '20,000-24,999': '09',
                "15,000-19,999": '08', "10,000-14,999": '07', "7,500-9,999": '06', '5,000-7,499': '05',
                "4,000-4,999": '04', "3,000-3,999": '03', "2,000-2,999": '02', '1,000-1,999': '01', 
                '$0-999': '00'
                 }
df_pay['rank'] = df_pay['rank'].replace(pay_sort_map)
df_pay['pay_rank'] = df_pay['rank'].astype(str) + ' - ' + df_pay['pay']

df_pay

df_pay = df_pay.merge(df_country[['Country', 'Count']], left_on='country', right_on='Country',
          suffixes=('_pay_category', '_total'))
df_pay['pay_pct'] = round(100 * df_pay['Count_pay_category'] / df_pay['Count_total'],1)
df_pay

NameError: name 'df_pay' is not defined

In [60]:
bubble_plot_pay = alt.Chart(df_pay).mark_circle(
        cornerRadiusTopLeft=3,
        cornerRadiusTopRight=3
    ).encode(
        y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
        x=alt.X('pay_rank:O', title="Pay", axis=alt.Axis(grid=False, labelAngle=270)),
        color=alt.Color('pay_rank:O', legend=None, title=""),
        size=alt.Size('pay_pct:Q'),    
    ).properties(width=570, height=500,title={
    "text": ["Salaries"],
    "subtitle": ["Broken Down proportionally by Country"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })

bubble_plot_pay

NameError: name 'alt' is not defined

In [ ]:
##conclusion
